In [2]:
!pip install flask-cors pyngrok

In [3]:
from pyngrok import ngrok
!ngrok authtoken 2rvCt0O3LY1ZQFX20uFGGcG4qZW_76is4e1n5YmKH7kpfUoXm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import threading
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

# Sample data
books = [
    {"id": 1, "title": "The Great Gatsby", "author": "F. Scott Fitzgerald"},
    {"id": 2, "title": "1984", "author": "George Orwell"}
]

@app.route('/')
def home():
    return "Hello from Flask API"

@app.route('/api/books', methods=['GET'])
def get_books():
    return jsonify(books)

@app.route('/api/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book is None:
        return jsonify({"error": "Book not found"}), 404
    return jsonify(book)

@app.route('/api/books', methods=['POST'])
def create_book():
    if not request.is_json:
        return jsonify({"error": "Content-Type must be application/json"}), 400

    data = request.get_json()

    if not all(key in data for key in ['title', 'author']):
        return jsonify({"error": "Title and author are required"}), 400

    new_id = max(book['id'] for book in books) + 1
    new_book = {
        'id': new_id,
        'title': data['title'],
        'author': data['author']
    }

    books.append(new_book)
    return jsonify(new_book), 201

# Start ngrok when the app starts
def run_ngrok():
    public_url = ngrok.connect(5000)
    print(f" * Public URL: {public_url}")

if __name__ == '__main__':
    # Install ngrok and flask-cors if you haven't already
    # !pip install flask-cors pyngrok

    # Start ngrok in a new thread
    threading.Thread(target=run_ngrok).start()

    # Run the Flask application
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 * Public URL: NgrokTunnel: "https://a9d7-34-106-238-4.ngrok-free.app" -> "http://localhost:5000"


INFO:werkzeug:127.0.0.1 - - [21/Jan/2025 03:37:30] "OPTIONS /api/books HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2025 03:37:31] "POST /api/books HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2025 03:38:48] "POST /api/books HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2025 03:39:06] "POST /api/books HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2025 03:39:55] "POST /api/books HTTP/1.1" 201 -
